# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 

import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list[0])

/home/workspace
/home/workspace/event_data/2018-11-27-events.csv


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data 
# into the Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [4]:
# checking the number of rows in your csv file

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# connection to a Cassandra instance on local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a Keyspace

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set the specified Keyspace

try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Description: 

## First I created the table and named it song_session because we are getting the
## song(s) that was played in a particular session. 
## The columns we need for this table are session_id, itemInSession, the name of the artist, 
## the title of the song, and the duration of the song.

query = "CREATE TABLE IF NOT EXISTS song_session "
query = query + """
                (
                sessionId int, 
                itemInSession int, 
                artist text, 
                song text, 
                length float, 
                PRIMARY KEY (sessionId, itemInSession)
                )
                """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    
    ## The rows matching each of the above specified columns were inserted into the 
    ## song_session table. Their corresponding column indexes in the 
    ## event_datafile_new.csv were specified for correct insertion
    
        query = """
                INSERT into song_session 
                (
                sessionId, 
                itemInSession, 
                artist, 
                song, 
                length
                )
                """
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Here I selected the columns: name of the artist, the title and length of the song, the session_id 
## and the item_in_session fro the song_session table where the session_id is 338 and the item_in_session 
## is 4.

query = """
        SELECT artist, song, length, sessionId, itemInSession 
        FROM song_session 
        WHERE sessionId = 338 and itemInSession = 4
        """

rows = session.execute(query)    

for row in rows:
    print (row.artist, "||", row.song, "||", row.length, "||", row.sessionid, "||", row.iteminsession)
    

Faithless || Music Matters (Mark Knight Dub) || 495.30731201171875 || 338 || 4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [25]:
## Description: 
## I created the table song_playlist_session because I want to get songs, artist that played for a 
## particular user in a session_id and the user details that played these songs. 

## The columns we need for this table are user_id, session_id, iteminsession, the name of the artist, 
## the title of the song, and the first and last names of the user.

query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + """
                (
                userId int, 
                sessionId int, 
                itemInSession int, 
                artist text, 
                song text, 
                firstName text, 
                lastName text, 
                PRIMARY KEY ((userId, sessionId), itemInSession))
                """
try:
    session.execute(query)
except Exception as e:
    print(e)


In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    
    ## The rows matching each of the above specified columns were inserted into the 
    ## song_session table. Their corresponding column indexes in the event_datafile_new.csv were specified 
    ## for correct insertion and the data types were also included.
    
        query = """
                INSERT into song_playlist_session 
                (
                userId, 
                sessionId, 
                itemInSession, 
                artist, 
                song, 
                firstName, 
                lastName
                )
                """
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],  line[1], line[4])) 


In [27]:
## Here, the query specified is to select the artist name, itemInSession, song title, first and last names 
## from the song_playlist_session the userId is 10 and sessionId is 182.

query2 = """
        SELECT artist, itemInSession, song, firstName, lastName 
        FROM song_playlist_session 
        WHERE userId = 10 and sessionId = 182
        """

rows = session.execute(query2)    

for row in rows:
    print (row.iteminsession, "||", row.artist, "||", row.song, "||", row.firstname, row.lastname)

0 || Down To The Bone || Keep On Keepin' On || Sylvie Cruz
1 || Three Drives || Greece 2000 || Sylvie Cruz
2 || Sebastien Tellier || Kilometer || Sylvie Cruz
3 || Lonnie Gordon || Catch You Baby (Steve Pitron & Max Sanna Radio Edit) || Sylvie Cruz


In [28]:
## Description: 
## I created the table song_listeners because I want to get the details of users that listened to the song
## 'All Hands Against His Own'

## The columns we need for this table are user_id, the song title, the first and last names of the user.
## The primary key used was the song and the userId because this will uniquely identify the rows. We need 
## the particular song ("All Hands Agaist His own") so the Primary key will be the song and the userId which
## will make the primary key unique.

query = "CREATE TABLE IF NOT EXISTS song_listeners "
query = query + """
                (
                song text,
                userId int, 
                firstName text, 
                lastName text, 
                PRIMARY KEY (song, userId)
                )
                """
try:
    session.execute(query)
except Exception as e:
    print(e)


In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        ## The rows matching each of the above specified columns were inserted into the 
        ## song_listeners table. Their corresponding column indexes in the event_datafile_new.csv 
        ## were specified for correct insertion and the data types were also included.
        
        query = """
                INSERT into song_listeners 
                (
                song, 
                userId, 
                firstName, 
                lastName
                )
                """
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))        


In [33]:
## Here, the query specified to select the first and last names 
## from the song_listeners table where the song title is "All Hands Against His Own".

query3 = """
        SELECT firstName, lastName 
        FROM song_listeners 
        WHERE song = 'All Hands Against His Own'
        """
rows = session.execute(query3)    

for row in rows:
    print (row.firstname, "||", row.lastname)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

### Drop the tables before closing out the sessions

In [21]:
## The tables are dropped here

In [22]:
query = "DROP TABLE IF EXISTS song_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query1 = "DROP TABLE IF EXISTS song_playlist_session"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "DROP TABLE IF EXISTS song_listeners"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()